<a href="https://colab.research.google.com/github/vedantichavan/Mini-Tasks-/blob/main/Pricient_Village.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import json
import random
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
village_menu = {
    "Rasam": 4.95,
    "Sambhar": 4.95,
    "Mulligatawny Soup": 5.49,
    "Garden Soup": 5.49,
    "Veg Manchow Soup": 5.95,
    "Tomato Soup": 5.25,
    "Idly": 5.95,
    "Mini Idly": 5.95,
    "Masala Idly": 7.95,
    "Ghee Fried Idly": 7.95,
    "Chilli Idly": 7.95,
    "Medu Vada": 5.95,
    "Medu Vada in Sambar Bowl": 5.99,
    "Medu Vada in Rasam Bowl": 5.99,
    "Dahi Vada": 4.95
}

In [ ]:
competitors_menu = [
    {"name": "Restaurant A", "menu": {
        "Rasam": 4.49, "Sambhar": 4.79, "Mulligatawny Soup": 5.29, "Garden Soup": 5.29,
        "Veg Manchow Soup": 5.79, "Tomato Soup": 5.19, "Idly": 5.49, "Mini Idly": 5.49,
        "Masala Idly": 7.49, "Ghee Fried Idly": 7.49, "Chilli Idly": 7.49, "Medu Vada": 5.79,
        "Medu Vada in Sambar Bowl": 5.89, "Medu Vada in Rasam Bowl": 5.89, "Dahi Vada": 4.79}},
    {"name": "Restaurant B", "menu": {
        "Rasam": 5.19, "Sambhar": 4.99, "Mulligatawny Soup": 5.59, "Garden Soup": 5.59,
        "Veg Manchow Soup": 6.19, "Tomato Soup": 5.49, "Idly": 5.99, "Mini Idly": 5.99,
        "Masala Idly": 8.19, "Ghee Fried Idly": 8.19, "Chilli Idly": 8.19, "Medu Vada": 6.19,
        "Medu Vada in Sambar Bowl": 6.29, "Medu Vada in Rasam Bowl": 6.29, "Dahi Vada": 5.09}},
    {"name": "Restaurant C", "menu": {
        "Rasam": 5.49, "Sambhar": 5.29, "Mulligatawny Soup": 5.99, "Garden Soup": 5.99,
        "Veg Manchow Soup": 6.49, "Tomato Soup": 5.79, "Idly": 6.49, "Mini Idly": 6.49,
        "Masala Idly": 8.49, "Ghee Fried Idly": 8.49, "Chilli Idly": 8.49, "Medu Vada": 6.49,
        "Medu Vada in Sambar Bowl": 6.59, "Medu Vada in Rasam Bowl": 6.59, "Dahi Vada": 5.29}}
]

In [ ]:
# Part 1 - Find the lowest price for each item
def get_lowest_prices(village_menu, competitors_menu):
    lowest_prices = {}
    for item in village_menu:
        # Get lowest price from competitors for each item
        lowest_price = min(
            restaurant["menu"].get(item, float('inf'))
            for restaurant in competitors_menu
        )
        lowest_prices[item] = lowest_price
    return lowest_prices

# Part 2 - Adjust prices based on busy times and weather
def adjust_prices_based_on_conditions(menu, lowest_prices, is_busy, temperature, weather_condition):
    adjusted_prices = {}
    for item, base_price in menu.items():
        # Adjust conditions
        is_cold = temperature < 45
        is_bad_weather = weather_condition in ["Rain", "Snow"]

        if is_busy or is_cold or is_bad_weather:
            # Higher price if conditions are met
            adjusted_prices[item] = lowest_prices[item] * 1.2  # 20% markup
        else:
            # Set to lowest competitive price
            adjusted_prices[item] = lowest_prices[item]
    return adjusted_prices

In [ ]:
# Simulated weather data and busyness
def simulate_conditions():
    temperature = random.randint(30, 80)  # Random temperature in Fahrenheit
    weather_condition = random.choice(["Clear", "Rain", "Snow"])  # Random weather
    is_busy = random.choice([True, False])  # Random busy state
    return temperature, weather_condition, is_busy

In [ ]:
# Part 3 - Machine Learning Pricing Prediction
def train_ml_model(village_menu, lowest_prices, competitors_menu):
    X = []
    y = []
    for _ in range(1000):  # Simulate 1000 data points
        temp, weather, busy = simulate_conditions()
        for item in village_menu:
            lowest_price = lowest_prices[item]
            base_price = village_menu[item]
            features = [
                temp,  # Temperature
                int(weather == "Rain"),  # Rainy
                int(weather == "Snow"),  # Snowy
                int(busy),  # Busy state
                lowest_price,  # Lowest competitor price
            ]
            price = adjust_prices_based_on_conditions(
                {item: base_price},
                {item: lowest_price},
                busy,
                temp,
                weather,
            )[item]
            X.append(features)
            y.append(price)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model with improved tree depth to avoid overfitting or underfitting
    model = DecisionTreeRegressor(max_depth=5, random_state=42)
    model.fit(X_train, y_train)

    # Test model
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Model Mean Squared Error: {mse}")

    return model


In [ ]:
# Main execution
def main():
    # Find lowest competitive prices
    lowest_prices = get_lowest_prices(village_menu, competitors_menu)
    print("Lowest competitive prices:", lowest_prices)

    # Simulate conditions and adjust prices
    temp, weather, busy = simulate_conditions()
    print(f"Simulated conditions - Temperature: {temp}°F, Weather: {weather}, Busy: {busy}")
    adjusted_prices = adjust_prices_based_on_conditions(village_menu, lowest_prices, busy, temp, weather)
    print("Adjusted prices based on conditions:", adjusted_prices)

    # Train ML model and show predictions
    model = train_ml_model(village_menu, lowest_prices, competitors_menu)
    test_conditions = [45, 1, 0, 1, 4.49]  # Example test conditions: 45°F, Rainy, Not Snowy, Busy, Lowest Price for Rasam
    for item in village_menu:
        predicted_price = model.predict([test_conditions])[0]
        print(f"Predicted price for {item}: ${predicted_price:.2f}")

if __name__ == "__main__":
    main()

Lowest competitive prices: {'Rasam': 4.49, 'Sambhar': 4.79, 'Mulligatawny Soup': 5.29, 'Garden Soup': 5.29, 'Veg Manchow Soup': 5.79, 'Tomato Soup': 5.19, 'Idly': 5.49, 'Mini Idly': 5.49, 'Masala Idly': 7.49, 'Ghee Fried Idly': 7.49, 'Chilli Idly': 7.49, 'Medu Vada': 5.79, 'Medu Vada in Sambar Bowl': 5.89, 'Medu Vada in Rasam Bowl': 5.89, 'Dahi Vada': 4.79}
Simulated conditions - Temperature: 54°F, Weather: Clear, Busy: True
Adjusted prices based on conditions: {'Rasam': 5.388, 'Sambhar': 5.748, 'Mulligatawny Soup': 6.348, 'Garden Soup': 6.348, 'Veg Manchow Soup': 6.9479999999999995, 'Tomato Soup': 6.228000000000001, 'Idly': 6.588, 'Mini Idly': 6.588, 'Masala Idly': 8.988, 'Ghee Fried Idly': 8.988, 'Chilli Idly': 8.988, 'Medu Vada': 6.9479999999999995, 'Medu Vada in Sambar Bowl': 7.068, 'Medu Vada in Rasam Bowl': 7.068, 'Dahi Vada': 5.748}
Model Mean Squared Error: 0.07382790580773102
Predicted price for Rasam: $5.39
Predicted price for Sambhar: $5.39
Predicted price for Mulligatawny S